# Welche Dateitypen gibt es?

## 1. Verbindung zur Datenbank
Es wird eine Verbindung zur Neo4j-Datenbank aufgebaut.

In [2]:
import py2neo

graph = py2neo.Graph(bolt=True, host='localhost', user='neo4j', password='neo4j')

## 2. Cypher-Abfrage
Es wird eine Abfrage an die Datenbank gestellt. Das Ergebnis wird in einem Dataframe (pandas) gespeichert.

In [3]:
import pandas as pd

query ="MATCH (f:Git:File) RETURN f.relativePath as relativePath"
df = pd.DataFrame(graph.run(query).data())


## 3. Datenaufbereitung
Zur Kontrolle werden die ersten fünf Zeilen des Ergebnisses der Abfrage als Tabelle ausgegeben.

In [4]:
df.head()

,relativePath
0,None
1,src/test/java/junit/samples/ListTest.java
2,src/main/java/org/junit/rules/ExpectedExceptio...
3,src/test/java/org/junit/rules/ExpectedExceptio...
4,src/test/java/org/junit/tests/running/methods/...


Der folgende Codeabschnitt extrahiert die verschiedenen Dateitypen entsprechend der Dateiendung und zählt deren Häufigkeit. Die Dateitypen werden in der Variable <font face="Courier">datatype</font> und deren Häufigkeit in der Variable <font face="Courier">frequency</font> gespeichert.

In [48]:

data=df['relativePath'].tolist()
liste=[]

for element in data:
    if element == None :                                #weil erster typ none typ war -> hat nicht funktioniert
        data.remove(element)

for element in data:
    s=element.split('.')[-1]       #extrahieren der dateitypen
    liste.append(s)

for element in liste:
    if len(element)>=6:
        liste.remove(element)




datatype=[]
for f in liste:
    if f not in datatype:
        datatype.append(f)
for element in datatype:        #zwar wiederholung aber ein langes war noch drin
    if len(element)>=6:
        datatype.remove(element)
print(datatype)

number=[]
n=''
for g in datatype:
    c=liste.count(g)
    n+=str(c)+' '
frequency=n.split(' ')
print(frequency)


['java', 'yml', 'xml', 'md', 'txt', 'cmd', 'jar', 'mvnw', 'fml', 'html', 'prefs', 'sh', 'svg', 'htm', 'pl', 'js', 'css', 'rb', 'gif', 'png', 'asc', 'ant', 'zip', 'txt~', 'GIF', 'arx', 'class']
['1383', '1', '14', '17', '21', '1', '19', '1', '1', '80', '2', '6', '1', '11', '1', '3', '6', '1', '36', '1', '1', '9', '2', '1', '8', '1', '41', '']


## 4. Visualisierung
Die Daten werden mittels eines Pie Charts visualisiert (https://plot.ly/python/pie-charts/, labels=datatype, values=frequency).

## 4. Visualisierung
Die Daten werden mittels eines Pie Charts visualisiert (https://plot.ly/python/pie-charts/, labels=datatype, values=frequency).



In [49]:
import plotly.plotly as py
import plotly.graph_objs as go

labels = datatype
values = frequency

trace = go.Pie(labels=labels, values=values)

py.iplot([trace], filename='basic_pie_chart')